In [5]:
from bs4 import BeautifulSoup
import requests

r = requests.get("https://www.transtats.bts.gov/Data_Elements.aspx?Qn6n=E", verify = False)
soup = BeautifulSoup(r.content, 'html.parser')
table_found = soup.find_all(['table'])[10]

In [2]:
import pandas as pd

airtraf = pd.read_html(str(table_found))
airtraf = airtraf[0]
airtraf.drop(airtraf[airtraf['Month'] == "TOTAL"].index, inplace = True)
airtraf.drop(airtraf[airtraf['Year'] < 2010].index, inplace = True)
airtraf = airtraf.reset_index()
del airtraf['index']

airtraf.head(12)

,Year,Month,DOMESTIC,INTERNATIONAL,TOTAL
0,2010,1,45501620,12393439,57895059
1,2010,2,42440614,10694165,53134779
2,2010,3,54424077,13279320,67703397
3,2010,4,52498074,12398700,64896774
4,2010,5,53842422,13380664,67223086
5,2010,6,56689853,14406776,71096629
6,2010,7,59142493,16026852,75169345
7,2010,8,57370748,15317926,72688674
8,2010,9,50304708,12599149,62903857
9,2010,10,54826050,12874707,67700757


In [3]:
import numpy as np
import sqlalchemy as sqla
import requests_cache
import plotnine as p9
requests_cache.install_cache('covidcast_cache')
import os, json
import glob
from scipy.stats import spearmanr
import matplotlib
import matplotlib.pyplot as plt

r_data_csv = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')
r_data_csv = r_data_csv[r_data_csv['Country_code'].isin(['US'])]
r_data_csv = r_data_csv.drop(['Country','WHO_region'],axis=1)
r_data_csv.set_index('Date_reported', inplace=True)
r_data_csv.index = pd.to_datetime(r_data_csv.index)
r_data_csv = r_data_csv.resample('1M').sum()
r_data_csv


,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
Date_reported,,,,
2020-01-31,47,354,0,0
2020-02-29,79,2328,1,1
2020-03-31,145687,801004,2643,13399
2020-04-30,862587,17730878,56229,856114
2020-05-31,731931,43648177,45098,2643117
2020-06-30,813512,63128418,23765,3484243
2020-07-31,1864090,107634613,27135,4349586
2020-08-31,1508065,162644591,30939,5318293
2020-09-30,1184379,195820984,21543,5916040
